# Retrieval-Augmented Generation: Question Answering based on Custom Dataset with Open-sourced [LangChain](https://python.langchain.com/en/latest/index.html) Library

Many use cases such as building a chatbot require text (text2text) generation models like **[BloomZ 7B1](https://huggingface.co/bigscience/bloomz-7b1)**, **[Flan T5 XXL](https://huggingface.co/google/flan-t5-xxl)**, and **[Flan T5 UL2](https://huggingface.co/google/flan-ul2)** to respond to user questions with insightful answers. The **BloomZ 7B1**, **Flan T5 XXL**, and **Flan T5 UL2** models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate how to use **BloomZ 7B1**, **Flan T5 XXL**, and **Flan T5 UL2** to answer questions using a library of documents as a reference, by using document embeddings and retrieval. The embeddings are generated from **GPT-J-6B** embedding model. 

**This notebook serves a template such that you can easily replace the example dataset by your own to build a custom question and asnwering application.**

## Step 1. Deploy large language model (LLM) and embedding model in SageMaker JumpStart

To better illustrate the idea, let's first deploy all the models that are required to perform the demo. You can choose either deploying all three Flan T5 XL, BloomZ 7B1, and Flan UL2 models as the large language model (LLM) to compare their model performances, or select **subset** of the models based on your preference. To do that, you need modify the `_MODEL_CONFIG_` python dictionary defined as below.

In [3]:
#!pip install --upgrade sagemaker --quiet
# !pip install ipywidgets==7.0.0 --quiet
# !pip install langchain --quiet
# !pip install faiss-cpu --quiet

%pip install -Uq \
    torch==1.13.1 \
    transformers==4.27.2 \
    langchain==0.0.143 \
    faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [6]:
import time
#import sagemaker, boto3, 
# from sagemaker.session import Session
# from sagemaker.model import Model
# from sagemaker import image_uris, model_uris, script_uris, hyperparameters
# from sagemaker.predictor import Predictor
# from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
import json
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

In [ ]:
# def query_endpoint_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
#     client = boto3.client("runtime.sagemaker")
#     response = client.invoke_endpoint(
#         EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
#     )
#     return response


# def parse_response_model_flan_t5(query_response):
#     model_predictions = json.loads(query_response["Body"].read())
#     generated_text = model_predictions["generated_texts"]
#     return generated_text


# def parse_response_multiple_texts_bloomz(query_response):
#     generated_text = []
#     model_predictions = json.loads(query_response["Body"].read())
#     for x in model_predictions[0]:
#         generated_text.append(x["generated_text"])
#     return generated_text

Deploy SageMaker endpoint(s) for large language models and GPT-J 6B embedding model. Please uncomment the entries as below if you want to deploy multiple LLM models to compare their performance.

In [ ]:
# _MODEL_CONFIG_ = {
#     "huggingface-text2text-flan-t5-xxl": {
#         "instance type": "ml.g5.12xlarge",
#         "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
#         "parse_function": parse_response_model_flan_t5,
#         "prompt": """Answer based on context:\n\n{context}\n\n{question}""",
#     },
#     "huggingface-textembedding-gpt-j-6b": {
#         "instance type": "ml.g5.24xlarge",
#         "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
#     },
#     # "huggingface-textgeneration1-bloomz-7b1-fp16": {
#     #     "instance type": "ml.g5.12xlarge",
#     #     "env": {},
#     #     "parse_function": parse_response_multiple_texts_bloomz,
#     #     "prompt": """question: \"{question}"\\n\nContext: \"{context}"\\n\nAnswer:""",
#     # },
#     # "huggingface-text2text-flan-ul2-bf16": {
#     #     "instance type": "ml.g5.24xlarge",
#     #     "env": {"TS_DEFAULT_WORKERS_PER_MODEL": "1"},
#     #     "parse_function": parse_response_model_flan_t5,
#     #     "prompt": """Answer based on context:\n\n{context}\n\n{question}""",
#     # },
# }

In [ ]:
# newline, bold, unbold = "\n", "\033[1m", "\033[0m"

# for model_id in _MODEL_CONFIG_:
#     endpoint_name = name_from_base(f"RAGResearch-{model_id}")
#     inference_instance_type = _MODEL_CONFIG_[model_id]["instance type"]

#     # Retrieve the inference container uri. This is the base HuggingFace container image for the default model above.
#     deploy_image_uri = image_uris.retrieve(
#         region=None,
#         framework=None,  # automatically inferred from model_id
#         image_scope="inference",
#         model_id=model_id,
#         model_version=model_version,
#         instance_type=inference_instance_type,
#     )
#     # Retrieve the model uri.
#     model_uri = model_uris.retrieve(
#         model_id=model_id, model_version=model_version, model_scope="inference"
#     )
#     model_inference = Model(
#         image_uri=deploy_image_uri,
#         model_data=model_uri,
#         role=aws_role,
#         predictor_cls=Predictor,
#         name=endpoint_name,
#         env=_MODEL_CONFIG_[model_id]["env"],
#     )
#     model_predictor_inference = model_inference.deploy(
#         initial_instance_count=1,
#         instance_type=inference_instance_type,
#         predictor_cls=Predictor,
#         endpoint_name=endpoint_name,
#     )
#     print(f"{bold}Model {model_id} has been deployed successfully.{unbold}{newline}")
#     _MODEL_CONFIG_[model_id]["endpoint_name"] = endpoint_name

## Step 2. Ask a question to LLM without providing the context

To better illustrate why we need retrieval-augmented generation (RAG) based approach to solve the question and anwering problem. Let's directly ask the model a question and see how they respond.

In [ ]:
# question = "Which instances can I use with Managed Spot Training in SageMaker?"

In [ ]:
# payload = {
#     "text_inputs": question,
#     "max_length": 100,
#     "num_return_sequences": 1,
#     "top_k": 50,
#     "top_p": 0.95,
#     "do_sample": True,
# }

# list_of_LLMs = list(_MODEL_CONFIG_.keys())
# list_of_LLMs.remove("huggingface-textembedding-gpt-j-6b")  # remove the embedding model


# for model_id in list_of_LLMs:
#     endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]
#     query_response = query_endpoint_with_json_payload(
#         json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
#     )
#     generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
#     print(f"For model: {model_id}, the generated output is: {generated_texts[0]}\n")

You can see the generated answer is wrong or doesn't make much sense. 

## Step 3. Improve the answer to the same question using **prompt engineering** with insightful context


To better answer the question well, we provide extra contextual information, combine it with a prompt, and send it to model together with the question. Below is an example.

In [ ]:
# context = """Managed Spot Training can be used with all instances supported in Amazon SageMaker. Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available."""

In [ ]:
# parameters = {
#     "max_length": 200,
#     "num_return_sequences": 1,
#     "top_k": 250,
#     "top_p": 0.95,
#     "do_sample": False,
#     "temperature": 1,
# }

# for model_id in list_of_LLMs:
#     endpoint_name = _MODEL_CONFIG_[model_id]["endpoint_name"]

#     prompt = _MODEL_CONFIG_[model_id]["prompt"]

#     text_input = prompt.replace("{context}", context)
#     text_input = text_input.replace("{question}", question)
#     payload = {"text_inputs": text_input, **parameters}

#     query_response = query_endpoint_with_json_payload(
#         json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
#     )
#     generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
#     print(
#         f"{bold}For model: {model_id}, the generated output is: {generated_texts[0]}{unbold}{newline}"
#     )

The output from step 3 tells us the chance to get the correct response significantly correlates with the insightful context you send into the LLM. 

**<span style="color:red">Now, the question becomes where can I find the insightful context based on the user query? The answer is to use a pre-stored knowledge data base with retrieval augmented generation, as shown in step 4 below</span>.**

## Step 4. Use RAG based approach with [LangChain](https://python.langchain.com/en/latest/index.html) and SageMaker endpoints to build a simplified question and answering application.


We plan to use document embeddings to fetch the most relevant documents in our document knowledge library and combine them with the prompt that we provide to LLM.

To achieve that, we will do following.

1. **Generate embedings for each of document in the knowledge library with SageMaker GPT-J-6B embedding model.**
2. **Identify top K most relevant documents based on user query.**
    - 2.1 **For a query of your interest, generate the embedding of the query using the same embedding model.**
    - 2.2 **Search the indexes of top K most relevant documents in the embedding space using in-memory Faiss search.**
    - 2.3 **Use the indexes to retrieve the corresponded documents.**
3. **Combine the retrieved documents with prompt and question and send them into SageMaker LLM.**



Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt -- maximum sequence length of 1024 tokens. 

---
To build a simiplied QA application with LangChain, we need: 
1. Wrap up our SageMaker endpoints for embedding model and LLM into `langchain.embeddings.SagemakerEndpointEmbeddings` and `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. That requires a small overwritten of `SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding mdoel.
2. Prepare the dataset to build the knowledge data base. 

---

Wrap up our SageMaker endpoints for embedding model into `langchain.embeddings.SagemakerEndpointEmbeddings`. That requires a small overwritten of `SagemakerEndpointEmbeddings` class to make it compatible with SageMaker embedding mdoel.

In [ ]:
# class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
#     def embed_documents(self, texts: List[str], chunk_size: int = 5) -> List[List[float]]:
#         """Compute doc embeddings using a SageMaker Inference Endpoint.

#         Args:
#             texts: The list of texts to embed.
#             chunk_size: The chunk size defines how many input texts will
#                 be grouped together as request. If None, will use the
#                 chunk size specified by the class.

#         Returns:
#             List of embeddings, one for each text.
#         """
#         results = []
#         _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size

#         for i in range(0, len(texts), _chunk_size):
#             response = self._embedding_func(texts[i : i + _chunk_size])
#             print
#             results.extend(response)
#         return results


# class ContentHandler(ContentHandlerBase):
#     content_type = "application/json"
#     accepts = "application/json"

#     def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
#         input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
#         return input_str.encode("utf-8")

#     def transform_output(self, output: bytes) -> str:
#         response_json = json.loads(output.read().decode("utf-8"))
#         embeddings = response_json["embedding"]
#         if len(embeddings) == 1:
#             return embeddings[0]
#         return embeddings


# content_handler = ContentHandler()

# embeddings = SagemakerEndpointEmbeddingsJumpStart(
#     endpoint_name=_MODEL_CONFIG_["huggingface-textembedding-gpt-j-6b"]["endpoint_name"],
#     region_name=aws_region,
#     content_handler=content_handler,
# )

Next, we wrap up our SageMaker endpoints for LLM into `langchain.llms.sagemaker_endpoint.SagemakerEndpoint`. 

In [ ]:
# from langchain.llms.sagemaker_endpoint import SagemakerEndpoint

# parameters = {
#     "max_length": 200,
#     "num_return_sequences": 1,
#     "top_k": 250,
#     "top_p": 0.95,
#     "do_sample": False,
#     "temperature": 1,
# }


# class ContentHandler(ContentHandlerBase):
#     content_type = "application/json"
#     accepts = "application/json"

#     def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
#         input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
#         return input_str.encode("utf-8")

#     def transform_output(self, output: bytes) -> str:
#         response_json = json.loads(output.read().decode("utf-8"))
#         return response_json["generated_texts"][0]


# content_handler = ContentHandler()

# sm_llm = SagemakerEndpoint(
#     endpoint_name=_MODEL_CONFIG_["huggingface-text2text-flan-t5-xxl"]["endpoint_name"],
#     region_name=aws_region,
#     model_kwargs=parameters,
#     content_handler=content_handler,
# )

Now, let's download the example data and prepare it for demonstration. We will use [Amazon SageMaker FAQs](https://aws.amazon.com/sagemaker/faqs/) as knowledge library. The data are formatted in a CSV file with two columns Question and Answer. We use the Answer column as the documents of knowledge library, from which relevant documents are retrieved based on a query. 

**For your purpose, you can replace the example dataset of your own to build a custom question and answering application.**

In [3]:
# original_data = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/"

# !mkdir -p rag_data
# !aws s3 cp --recursive $original_data rag_data

download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to rag_data/Amazon_SageMaker_FAQs.csv


For the case when you have data saved in multiple subsets. The following code will read all files that end with `.csv` and concatenate them together. Please ensure each `csv` file has the same format.

In [7]:
import glob
import os
import pandas as pd

all_files = glob.glob(os.path.join("data-rag/", "*.csv"))

df_knowledge = pd.concat(
    (pd.read_csv(f, header=None, names=["Question", "Answer"]) for f in all_files),
    axis=0,
    ignore_index=True,
)

Drop the `Question` column as it is not used in this demonstration.

In [8]:
df_knowledge.drop(["Question"], axis=1, inplace=True)

In [9]:
df_knowledge.head(5)

,Answer
0,Amazon SageMaker is a fully managed service to...
1,For a list of the supported Amazon SageMaker A...
2,Amazon SageMaker is designed for high availabi...
3,Amazon SageMaker stores code in ML storage vol...
4,Amazon SageMaker ensures that ML model artifac...


In [10]:
df_knowledge.to_csv("data-rag/processed_data.csv", header=False, index=False)

In [11]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders.csv_loader import CSVLoader

Use langchain to read the `csv` data. There are multiple built-in functions in LangChain to read different format of files such as `txt`, `html`, and `pdf`. For details, see [LangChain document loaders](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html).

In [14]:
loader = CSVLoader(file_path="data-rag/processed_data.csv")

In [15]:
documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
# texts = text_splitter.split_documents(documents) ### if you use langchain.document_loaders.TextLoader to load text file. You can uncomment the code
## to split the text.

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

Based on the question below, we can achieven the points in Step 4 with just a few lines of code as shown below.

In [17]:
#question = "Which instances can I use with Managed Spot Training in SageMaker?"
# context = """Managed Spot Training can be used with all instances supported in Amazon SageMaker. Managed Spot Training is supported in all AWS Regions where Amazon SageMaker is currently available."""

In [20]:
import transformers
import torch
import pandas as pd
import numpy as np

import json
import boto3
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5EncoderModel

# Note:  we need to use at least t5-large, otherwise the embeddings do not match the right document later.
#        it might be good to actually show this, maybe?  and show the similarity search results later?

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
encoder_model = T5EncoderModel.from_pretrained("google/flan-t5-large")

Some weights of the model checkpoint at google/flan-t5-large were not used when initializing T5EncoderModel: ['decoder.block.20.layer.1.EncDecAttention.v.weight', 'decoder.block.10.layer.1.EncDecAttention.q.weight', 'decoder.block.8.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.17.layer.0.layer_norm.weight', 'decoder.block.13.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.13.layer.2.DenseReluDense.wo.weight', 'decoder.block.18.layer.0.SelfAttention.k.weight', 'decoder.block.9.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.14.layer.0.layer_norm.weight', 'decoder.block.22.layer.0.SelfAttention.q.weight', 'decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.19.layer.0.SelfAttention.q.weight', 'decoder.block.6.layer.0.layer_norm.weight', 'decoder.block.10.layer.1.EncDecAttention.v.weight', 'decoder.block.11.layer.1.EncDecAttention.q.weight', 'decoder.block.19.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.13.layer.1.En

In [21]:
prompt = f'''Answer the following question. If you do not have the information to answer the question, say "I don't know".
Question: Who won the 2022 world cup?
Answer:
'''
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        input_ids=inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

print(f'{prompt}{output}')

Answer the following question. If you do not have the information to answer the question, say "I don't know".
Question: Who won the 2022 world cup?
Answer:
I don't know


# Get a HuggingFace Model for Embeddings

Load in the [FLAN T5 large model](https://huggingface.co/google/flan-t5-large) from HuggingFace. This will be the model we use to create our document search embeddings.

# Create Embedding Database

Now use the HuggingFace model to create embeddings for each of the three documents which have been provided. The documents used here are only illustrative. These documents could be extended to any collection of text to help supplement your use case.

In [22]:
import torch

def get_embedding(text, encoder_model, tokenizer):
    with torch.no_grad():
        input_ids = tokenizer(
            text, return_tensors="pt", max_length=512, truncation=True
        ).input_ids #.to(DEVICE)
        outputs = encoder_model(input_ids=input_ids)
        last_hidden_states = outputs.last_hidden_state
        e = last_hidden_states.mean(dim=1)
    return e

In [23]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=get_embedding(prompt, encoder_model, tokenizer),
    text_splitter=CharacterTextSplitter(chunk_size=300, chunk_overlap=0),
)

ValidationError: 1 validation error for VectorstoreIndexCreator
embedding
  instance of Embeddings expected (type=type_error.arbitrary_type; expected_arbitrary_type=Embeddings)

In [ ]:
index = index_creator.from_loaders([loader])

In [ ]:
index.query(question=question, llm=sm_llm)

## Step 5. Customize the QA application above with different prompt.

Now, we see how simple it is to use LangChain to achieve question and answering application with just few lines of code. Let's break down the above `VectorstoreIndexCreator` and see what's happening under the hood. Furthermore, we will see how to incorporate a customize prompt rather than using a default prompt with `VectorstoreIndexCreator`.

Firstly, we **generate embedings for each of document in the knowledge library with SageMaker GPT-J-6B embedding model.**

In [ ]:
docsearch = FAISS.from_documents(documents, embeddings)

In [ ]:
question

Based on the question above, we then **identify top K most relevant documents based on user query, where K = 3 in this setup**.

In [ ]:
docs = docsearch.similarity_search(question, k=3)

Print out the top 3 most relevant docuemnts as below.

In [ ]:
docs

Finally, we **combine the retrieved documents with prompt and question and send them into SageMaker LLM.** 

We define a customized prompt as below.

In [ ]:
prompt_template = """Answer based on context:\n\n{context}\n\n{question}"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [ ]:
chain = load_qa_chain(llm=sm_llm, prompt=PROMPT)

Send the top 3 most relevant docuemnts and question into LLM to get a answer.

In [ ]:
result = chain({"input_documents": docs, "question": question}, return_only_outputs=True)[
    "output_text"
]

Print the final answer from LLM as below, which is accurate.

In [ ]:
result